# Interventions

TBC

In [ ]:
import starsim as ss

def sis_vaccine(sim, start=2030, eff=0.9):
    if sim.now == start:
        sim.diseases.sis.rel_trans[:] *= 1-eff
    return

pars = dict(
    start=2000,
    stop=2050,
    diseases='sis',
    networks='random',
    interventions=sis_vaccine,
)

sim = ss.Sim(pars)
sim.run()
sim.plot()
ss.show()